<a href="https://colab.research.google.com/github/Vivek-afk81/data_mining/blob/main/clustering_techniques.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/My Drive/data_mining')
print(os.getcwd())

/content/drive/My Drive/data_mining


##k-NEAREST NEIGHBORS


###Import libraries


In [3]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import ConfusionMatrixDisplay

###Load and prepare the data


In [4]:
trn=pd.read_csv('data/spambase_trn.csv')
X_trn=trn.drop('y',axis=1)
y_trn=trn['y']
tst=pd.read_csv('data/spambase_tst.csv')
X_tst=tst.drop('y',axis=1)
y_tst=tst['y']

trn.head()

,X0,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X48,X49,X50,X51,X52,X53,X54,X55,X56,y
0,0.00,0.00,0.00,0.0,0.0,0.00,1.21,0.00,0.0,1.21,...,0.0,0.000,0.0,0.000,0.000,0.000,1.676,15,57,1
1,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.0,0.00,...,0.0,0.208,0.0,0.417,0.208,0.000,3.812,16,61,1
2,0.33,0.00,0.33,0.0,0.0,0.00,0.66,0.00,0.0,0.00,...,0.0,0.116,0.0,0.406,0.464,0.348,6.932,43,513,1
3,0.52,1.31,0.26,0.0,2.9,0.26,0.79,0.26,0.0,0.79,...,0.0,0.138,0.0,0.046,0.000,0.000,2.934,60,135,1
4,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.0,0.00,...,0.0,0.000,0.0,0.000,0.000,0.000,14.000,53,56,0


###k-NN : Train the model

In [5]:
knn=KNeighborsClassifier(
    n_neighbors=5
)
knn.fit(X_trn,y_trn)

KNeighborsClassifier()

###Calculate Mean Accuracy on the Training Data


In [6]:
print("Accuracy on the training data: "+str("{:.2%}".format(knn.score(X_trn,y_trn))))

Accuracy on the training data: 86.86%
